# Implementing Rate Limiting in FastAPI

In this tutorial, we'll explore how to implement rate limiting in a FastAPI application to control the number of requests a client can make within a specified time window. We'll cover the concepts behind rate limiting, different strategies for implementation, and provide step-by-step instructions with code examples. By the end of this tutorial, you'll be able to integrate rate limiting into your FastAPI applications effectively.

## Table of Contents

1. [Introduction](#1-introduction)
2. [Prerequisites](#2-prerequisites)
3. [Understanding Rate Limiting](#3-understanding-rate-limiting)
4. [Rate Limiting Strategies](#4-rate-limiting-strategies)
5. [Setting Up the Environment](#5-setting-up-the-environment)
6. [Installing Dependencies](#6-installing-dependencies)
7. [Creating the FastAPI Application](#7-creating-the-fastapi-application)
8. [Implementing Rate Limiting Using Middleware](#8-implementing-rate-limiting-using-middleware)
9. [Implementing Rate Limiting Using Dependencies](#9-implementing-rate-limiting-using-dependencies)
10. [Implementing Rate Limiting with Redis](#10-implementing-rate-limiting-with-redis)
11. [Advanced Rate Limiting Techniques](#11-advanced-rate-limiting-techniques)
    - [11.1. IP-Based Rate Limiting](#111-ip-based-rate-limiting)
    - [11.2. User-Based Rate Limiting](#112-user-based-rate-limiting)
    - [11.3. Endpoint-Specific Rate Limiting](#113-endpoint-specific-rate-limiting)
12. [Testing the Rate Limiter](#12-testing-the-rate-limiter)
13. [Conclusion](#13-conclusion)
14. [References](#14-references)

## 1. Introduction

**Rate limiting** is a technique used to control the rate at which clients can make requests to an API. It helps protect your application from abuse, prevents server overload, and ensures fair usage among clients.

In this tutorial, we'll:

- Understand the importance of rate limiting.
- Explore different strategies for implementing rate limiting in FastAPI.
- Implement rate limiting using middleware, dependencies, and Redis.
- Explore advanced techniques for more granular control.

## 2. Prerequisites

Before we begin, ensure you have the following:

- **Python 3.7+** installed.
- Basic knowledge of **Python**, **FastAPI**, and **asynchronous programming**.
- Familiarity with concepts like middleware and dependencies in FastAPI.

## 3. Understanding Rate Limiting

Rate limiting controls how many requests a client can make to an API within a specified time window. It helps:

- **Prevent Abuse**: Protects against denial-of-service (DoS) attacks.
- **Manage Resources**: Ensures the server isn't overwhelmed by too many requests.
- **Fair Usage**: Allows equitable distribution of resources among clients.

## 4. Rate Limiting Strategies

Common strategies for implementing rate limiting include:

- **Fixed Window**: Limits requests in fixed intervals (e.g., 100 requests per minute).
- **Sliding Window**: Uses a sliding time window for more granular control.
- **Token Bucket**: Allows bursts of requests up to a limit, then refills tokens over time.
- **Leaky Bucket**: Processes requests at a constant rate, queues excess requests.

We'll focus on the fixed window algorithm for simplicity.

## 5. Setting Up the Environment

Create a new project directory and set up a virtual environment.

```bash
# Create project directory
mkdir fastapi-rate-limit
cd fastapi-rate-limit

# Set up virtual environment
python -m venv venv
source venv/bin/activate  # On Windows: venv\Scripts\activate
```

## 6. Installing Dependencies

Install the necessary packages.

```bash
pip install fastapi uvicorn[standard] aioredis
```

- **fastapi**: The web framework.
- **uvicorn**: ASGI server.
- **aioredis**: Async Redis client (if using Redis for rate limiting).

## 7. Creating the FastAPI Application

**Directory Structure**

```
fastapi-rate-limit/
├── main.py
├── middleware.py
├── dependencies.py
├── requirements.txt
```

Create the files:

```bash
touch main.py middleware.py dependencies.py
```

## 8. Implementing Rate Limiting Using Middleware

One way to implement rate limiting is by creating a middleware that intercepts each request.

### 8.1. Creating the Middleware

**middleware.py**

```python
from starlette.middleware.base import BaseHTTPMiddleware
from starlette.responses import JSONResponse
from time import time

class RateLimitMiddleware(BaseHTTPMiddleware):
    def __init__(self, app, max_requests: int, window_seconds: int):
        super().__init__(app)
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.clients = {}

    async def dispatch(self, request, call_next):
        client_ip = request.client.host
        current_time = time()
        window = current_time // self.window_seconds

        if client_ip not in self.clients:
            self.clients[client_ip] = {}

        client_windows = self.clients[client_ip]

        if window not in client_windows:
            client_windows[window] = 0

        if client_windows[window] >= self.max_requests:
            return JSONResponse(
                status_code=429,
                content={"detail": "Too Many Requests"}
            )

        client_windows[window] += 1

        response = await call_next(request)
        return response
```

**Explanation**:

- **RateLimitMiddleware**: Inherits from `BaseHTTPMiddleware`.
- **clients**: Dictionary storing request counts per client IP and time window.
- **dispatch**: Checks if the client has exceeded the max requests in the current window.

### 8.2. Adding Middleware to the Application

**main.py**

```python
from fastapi import FastAPI
from middleware import RateLimitMiddleware

app = FastAPI()

# Configure rate limiting: e.g., 5 requests per 60 seconds
app.add_middleware(RateLimitMiddleware, max_requests=5, window_seconds=60)

@app.get("/")
async def root():
    return {"message": "Welcome to the rate-limited API!"}
```

## 9. Implementing Rate Limiting Using Dependencies

Alternatively, you can implement rate limiting using FastAPI dependencies.

### 9.1. Creating the Dependency

**dependencies.py**

```python
from fastapi import HTTPException, status, Request
from time import time
from functools import wraps

clients = {}

def rate_limit(max_requests: int, window_seconds: int):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            request = kwargs.get("request")
            if request is None:
                for arg in args:
                    if isinstance(arg, Request):
                        request = arg
                        break
            if request is None:
                raise ValueError("Request object not found in arguments")

            client_ip = request.client.host
            current_time = time()
            window = current_time // window_seconds

            if client_ip not in clients:
                clients[client_ip] = {}

            client_windows = clients[client_ip]

            if window not in client_windows:
                client_windows.clear()  # Clear old windows
                client_windows[window] = 0

            if client_windows[window] >= max_requests:
                raise HTTPException(
                    status_code=status.HTTP_429_TOO_MANY_REQUESTS,
                    detail="Too Many Requests"
                )

            client_windows[window] += 1

            return await func(*args, **kwargs)
        return wrapper
    return decorator
```

### 9.2. Applying the Dependency

**main.py**

```python
from fastapi import FastAPI, Request
from dependencies import rate_limit

app = FastAPI()

@app.get("/items/")
@rate_limit(max_requests=10, window_seconds=60)
async def read_items(request: Request):
    return {"message": "Here are your items"}
```

**Explanation**:

- The `rate_limit` decorator uses a closure to accept parameters.
- It tracks requests per client IP and time window.
- Raises `HTTPException` when the limit is exceeded.

## 10. Implementing Rate Limiting with Redis

For a production-ready solution, it's better to use an external datastore like Redis to track request counts, especially if you have multiple instances of your application.

### 10.1. Setting Up Redis

Install Redis (if not already installed). You can use Docker:

```bash
docker run -d --name redis -p 6379:6379 redis
```

Install `aioredis` if not already installed:

```bash
pip install aioredis
```

### 10.2. Implementing the Rate Limiter

**dependencies.py**

```python
import aioredis
from fastapi import HTTPException, Request, status

redis = aioredis.from_url("redis://localhost")

async def rate_limiter(request: Request, max_requests: int = 5, window_seconds: int = 60):
    client_ip = request.client.host
    key = f"rate-limit:{client_ip}"
    current = await redis.incr(key)

    if current == 1:
        await redis.expire(key, window_seconds)

    if current > max_requests:
        ttl = await redis.ttl(key)
        raise HTTPException(
            status_code=status.HTTP_429_TOO_MANY_REQUESTS,
            detail=f"Rate limit exceeded. Try again in {ttl} seconds."
        )
```

### 10.3. Applying the Dependency

**main.py**

```python
from fastapi import FastAPI, Request, Depends
from dependencies import rate_limiter

app = FastAPI()

@app.get("/users/")
async def get_users(request: Request, _: None = Depends(rate_limiter)):
    return {"message": "List of users"}
```

**Explanation**:

- **rate_limiter**: Uses Redis to keep track of request counts per client IP.
- **Key**: `rate-limit:<client_ip>` to uniquely identify clients.
- **Expire**: Sets the TTL for the key to the window size.
- **TTL**: Time until the rate limit resets.

## 11. Advanced Rate Limiting Techniques

### 11.1. IP-Based Rate Limiting

We already implemented IP-based rate limiting in the previous examples.

### 11.2. User-Based Rate Limiting

For authenticated users, you can limit requests based on user IDs instead of IP addresses.

**dependencies.py**

```python
async def rate_limiter_user(request: Request, user_id: str, max_requests: int = 10, window_seconds: int = 60):
    key = f"rate-limit:user:{user_id}"
    current = await redis.incr(key)

    if current == 1:
        await redis.expire(key, window_seconds)

    if current > max_requests:
        ttl = await redis.ttl(key)
        raise HTTPException(
            status_code=status.HTTP_429_TOO_MANY_REQUESTS,
            detail=f"Rate limit exceeded. Try again in {ttl} seconds."
        )
```

**main.py**

```python
from fastapi import Depends

# Assume we have a dependency that gets the current user
async def get_current_user():
    # Logic to get the current user
    return {"id": "user123"}

@app.get("/profile/")
async def get_profile(
    request: Request,
    current_user: dict = Depends(get_current_user),
    _: None = Depends(lambda: rate_limiter_user(request, current_user['id']))
):
    return {"message": "User profile"}
```

### 11.3. Endpoint-Specific Rate Limiting

You can specify different rate limits for different endpoints.

**main.py**

```python
@app.get("/public/")
@rate_limit(max_requests=100, window_seconds=60)
async def public_endpoint(request: Request):
    return {"message": "Public data"}

@app.get("/sensitive/")
@rate_limit(max_requests=5, window_seconds=60)
async def sensitive_endpoint(request: Request):
    return {"message": "Sensitive data"}
```

## 12. Testing the Rate Limiter

You can test the rate limiter using tools like **cURL** or **ApacheBench**.

### 12.1. Using a Python Script

```python
import requests

url = "http://localhost:8000/items/"
for i in range(15):
    response = requests.get(url)
    print(f"Request {i+1}: Status {response.status_code}, Response: {response.json()}")
```

### 12.2. Expected Output

For the endpoint with a limit of 10 requests per minute:

- The first 10 requests should return status code **200**.
- Subsequent requests should return status code **429** with a "Too Many Requests" message.

## 13. Conclusion

In this tutorial, we've:

- Understood the importance of rate limiting.
- Implemented rate limiting using middleware and dependencies in FastAPI.
- Utilized Redis for a scalable, production-ready rate limiting solution.
- Explored advanced techniques like user-based and endpoint-specific rate limiting.

Implementing rate limiting helps protect your API from abuse, ensures fair usage, and enhances the overall reliability of your application.

## 14. References

- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [Starlette Middleware](https://www.starlette.io/middleware/)
- [Redis Official Site](https://redis.io/)
- [aioredis Documentation](https://aioredis.readthedocs.io/en/latest/)
- [Rate Limiting Algorithms](https://en.wikipedia.org/wiki/Rate_limiting)